In [137]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [138]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [139]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [140]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
Xte,  Yte  = build_dataset(words[n2:])     # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [141]:
# utility function we will use later when comparing manual gradients to PyTorch gradients
def validate(s, dt, t):
  ex = torch.all(dt == t.grad).item()
  app = torch.allclose(dt, t.grad)
  maxdiff = (dt - t.grad).abs().max().item()
  print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [142]:
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 64 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1 # using b1 just for fun, it's useless because of BN
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

# Note: I am initializating many of these parameters in non-standard ways
# because sometimes initializating with e.g. all zeros could mask an incorrect
# implementation of the backward pass.

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

4137


In [143]:
batch_size = 32
n = batch_size # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [144]:
# forward pass, "chunkated" into smaller steps that are possible to backward one at a time

emb = C[Xb] # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1/n*hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# cross entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdims=True)
counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean() 

# PyTorch backward pass
for p in parameters:
  p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, # afaik there is no cleaner way
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss

tensor(3.3296, grad_fn=<NegBackward0>)

In [186]:
# Exercise 1: backprop through the whole thing manually, 
# backpropagating through exactly all of the variables 
# as they are defined in the forward pass above, one by one
# ---------------------------------------------------------

# loss = -logprobs[range(n), Yb].mean() 
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1/n
validate('logprobs', dlogprobs, logprobs)

# logprobs = probs.log()
# -> logprobs = log(probs)
# derivative of f'log(x) = 1/x
# -> f'log(probs) = 1/probs
# and then we just apply chain rule from "calculus" lol
# and chain rule is basically we apply multiplier from previous node, which is -1/n, which is dlogprobs
dprobs = (1 / probs) * dlogprobs
validate('probs', dprobs, probs)

# we need this sum since counst_sum_inv is just a scalar value in each batch
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
validate('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

# this we will use later, we don't need .sum here since counts_sum_inv affect dcounts once
dcounts = counts_sum_inv * dprobs

#counts_sum_inv = counts_sum**-1 = 1/counts_sum
# this one is pretty simple, d/dx of 1/x is -1/x^2 which is -x^-2, and then chain rule
dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
validate('counts_sum', dcounts_sum, counts_sum)

# counts_sum = counts.sum(1, keepdims=True)
# here we want to propagate global derivative 
# to each individual count, through each row
# because local derivative of each of them is 1, since its just sum operation aka "router"

reshaped_dcounts_sum = torch.ones_like(counts) * dcounts_sum
dcounts += reshaped_dcounts_sum 
validate('counts', dcounts, counts)

# counts = norm_logits.exp()
dnorm_logits = norm_logits.exp() * dcounts
validate('norm_logits', dnorm_logits, norm_logits)

# norm_logits = logits - logit_maxes # subtract max for numerical stability
dlogits = dnorm_logits.clone() * 1
dlogit_maxes = (-1 * dnorm_logits).sum(1, keepdim=True)
validate('logit_maxes', dlogit_maxes, logit_maxes)

# logit_maxes = logits.max(1, keepdim=True).values
# here we got boolean tensor, which is basically same shape as logits, but booleans
# and True will be only at indexes where logit_maxes appears, logit_maxes broadcasts here
mask = (logits == logit_maxes)
# after we got mask, we want to transform it to integet, or more specifically, one hot tensor
# but F.one_hot doesnt work with boolean tensors, so
dlogits += mask.int() * dlogit_maxes
validate('logits', dlogits, logits)

# logits = h @ W2 + b2 # output layer
# this entire thing is based on the fact that we multiply matricies
# and we can decompose it to the atoms and by doing that comeup with solutions below
# it's pretty verbose and tbh I didn't get it FULLY, but level of understanding is good enough
# also there are tricks to commup with right formulas:
# 1. keep in mind that formula should be matrix mul
# 2. dimensions should match
dh = dlogits @ W2.T
validate('h', dh, h)
dW2 = h.T @ dlogits
validate('W2', dW2, W2)
db2 = dlogits.sum(0)
validate('b2', db2, b2)

# h = torch.tanh(hpreact) # hidden layer
# easy. y = tanh(x), d/dtanh = 1 - y^2
# and as always, chain rule * dh
dhpreact = (1 - h**2) * dh
validate('hpreact', dhpreact, hpreact)

# hpreact = bngain * bnraw + bnbias
# this one should be easy peasy, want to solve it without cheating lol
# easy for dbngain, with trick that dimensions should match
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
validate('bngain', dbngain, bngain)
# same for this thing, gradient is 1 since we have plus op
# and than chain rule + match of the dimensions
dbnbias = dhpreact.sum(0, keepdim=True)
validate('bnbias', dbnbias, bnbias)

dbnraw = bngain * dhpreact # broadcasting will happen and we should be fine
validate('bnraw', dbnraw, bnraw)

# bnraw = bndiff * bnvar_inv
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
validate('bnvar_inv', dbnvar_inv, bnvar_inv)

dbndiff = bnvar_inv * dbnraw
# bnvar_inv = (bnvar + 1e-5)**-0.5
dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
validate('bnvar', dbnvar, bnvar)

# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# well i didn't really get math behind it, but we have a * b, and then sum
# which means on the backward pass we just interested in a and rest will be done by broadcasting
dbndiff2 = (1/(n-1))*torch.ones_like(bndiff2) * dbnvar
validate('bndiff2', dbndiff2, bndiff2)

# bndiff2 = bndiff**2
# power rule, + it's accumulated because we already encountered dbndiff
dbndiff += 2*bndiff*dbndiff2
validate('bndiff', dbndiff, bndiff)

# bndiff = hprebn - bnmeani
dbnmeani = (dbndiff * -1).sum(0, keepdim=True)
validate('bnmeani', dbnmeani, bnmeani)

# bndiff = hprebn - bnmeani
dhprebn = dbndiff
# # BatchNorm layer
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
dhprebn += 1/n * dbnmeani
validate('hprebn', dhprebn, hprebn)

# hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# same thing as before with fokn matricies
dembcat = dhprebn @ W1.T
validate('embcat', dembcat, embcat)
dW1 = embcat.T @ dhprebn
validate('W1', dW1, W1)
db1 = dhprebn.sum(0, keepdim=True)
validate('b1', db1, b1)

# embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
# smol amount idea, but first try after checking shapes
demb = (dhprebn @ W1.T).view(32,3,10)
validate('emb', demb, emb)

# emb = C[Xb] # embed the characters into vectors
# untackling and routing gradient to the initial vectorized representations of letters
dC = torch.zeros_like(C)
for i, example_row in enumerate(Xb):
    for j, idx in enumerate(example_row):
        f_gradient = demb[i, j]
        dC[idx] += f_gradient
        
validate('C', dC, C)



logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnbias          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff:

In [ ]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

In [193]:
# backward pass

# -----------------
# YOUR CODE HERE :)
# -----------------


# ==== 🔥 SOFTMAX 🔥 ====
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 
# probs = counts * counts_sum_inv
# ==== 🔥 SOFTMAX 🔥 ====
# same as: probs = exp(logits) / sum(exp(logits))

# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# didn't get it fully tbh
# there are knowing derivatives of softmax and we utilize them
# and then since we have not one thing, but butch, we do =/n to respect mean
dlogits = torch.softmax(logits, dim=1)
dlogits[torch.arange(n), Yb] -= 1
dlogits /= n

validate('logits', dlogits, logits) 

logits          | exact: False | approximate: True  | maxdiff: 6.984919309616089e-09
